In [ ]:
from bs4 import BeautifulSoup
import requests as rq
from unicodedata import normalize
import re

In [ ]:
url = "https://www.businessinsider.com/author/francis-agustin"

In [ ]:
response = rq.get(url)
page_source = BeautifulSoup(response.text, 'lxml')

In [ ]:
pos = page_source.find('p', attrs={'data-test': "author-aside-title"}).text.strip()
pos

In [ ]:
author_bio = [a.text.strip() for a in page_source.select('div.col-12.col-md-8.author-description > p')]
author_bio = normalize("NFKD", ''.join(map(str, author_bio)).replace('  ', ' ').strip())
author_bio

In [ ]:
# 'a.author-contact-icon-link.share-link.email::attr(href)'

author_email = page_source.find('a', attrs={'class': 'author-contact-icon-link share-link email'})['href'].replace('mailto:', '').strip()
author_email

In [ ]:
author_twitter = page_source.find('a', attrs={
                    'class': 'author-contact-icon-link share-link twitter'})['href'].strip()
author_twitter

In [ ]:
url = 'https://markets.businessinsider.com/news/currencies/stablecoin-regulation-cryptocurrency-banks-biden-treasury-bitcoin-circle-tether-binance-2021-10'

In [ ]:
response = rq.get(url)
page_source = BeautifulSoup(response.text, 'lxml')

In [ ]:
def remove_articles(text):
    # strip the unicode articles
    #text = normalize("NFKD", text.strip(u'\u201c'u'\u201d'))
    text = normalize("NFKD", ''.join(map(str, text)).replace('  ', ' ').strip())
    return text

from itemloaders.processors import Join, MapCompose
from scrapy.item import Item, Field

In [ ]:
proc = MapCompose(remove_articles)

class InsiderArtItem(Item):
    content = Field(
        input_processor=MapCompose(remove_articles),
        # TakeFirst return the first value not the whole list
        output_processor=Join()
        )

article = InsiderArtItem()

In [ ]:
article_content = [a.text.strip() for a in page_source.select(
    'div.col-xs-12.news-content.no-padding > p, div.col-xs-12.news-content.no-padding > ul > li[aria-level*="1"]')] # div.col-xs-12.news-content.no-padding > p > a,
article_content

In [ ]:
# article['content'] = article_content
# article['content']

# article['content'] = [remove_articles(line) for line in article_content]
# article['content'] = ' '.join(article['content'])
# article['content']

article_content = [remove_articles(a.text) for a in page_source.select(
    'div.col-xs-12.news-content.no-padding > p, div.col-xs-12.news-content.no-padding > ul > li[aria-level*="1"]')] # div.col-xs-12.news-content.no-padding > p > a,
article_content = ''.join(article_content)
article_content

## CNBC

In [ ]:
from bs4 import BeautifulSoup
# import requests as rq
import grequests
from unicodedata import normalize
import re

In [ ]:
urls =  ['https://www.cnbc.com/matthew-j-belvedere/']

In [ ]:
def get_urls(bio_links):
    reqs = [grequests.get(bio_link) for bio_link in bio_links]
    resps = grequests.map(reqs)
    return resps

def process_author(resps):
    auth  = 'Matthew J. Belvedere'
    authors = []
    authors = {}
    authors[f'{auth}'] = {}
    for (auth, response) in zip(authors.keys(), resps):
        page_source = BeautifulSoup(response.text, 'lxml')
        return page_source
        # pos = page_source.find('span', attrs={'class': "RenderBioDetails-jobTitle"}).text.strip()
        # print(pos)
        # authors[f'{auth}']['author_position'] = pos
        # return page_source, pos


In [ ]:
resp = get_urls(urls)
page_source  = process_author(resp)

In [ ]:
js_script = page_source.xpath('//script[contains(., "locationBeforeTransitions")]/text()')

In [ ]:
# page_source

In [ ]:
from bs4 import BeautifulSoup
from lxml import etree
import requests
from FiScrape.items import index_of_nth
import json
  
  
URL = "https://www.cnbc.com/matthew-j-belvedere/"
  
HEADERS = ({'User-Agent':
            "Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; compatible; Googlebot/2.1; \
            +http://www.google.com/bot.html) Chrome/W.X.Y.Z Safari/537.36",\
            'Accept-Language': 'en-US, en;q=0.5'})
            
webpage = requests.get(URL, headers=HEADERS)
soup = BeautifulSoup(webpage.content, "lxml")
dom = etree.HTML(str(soup))
js_script = dom.xpath('//script[contains(., "locationBeforeTransitions")]/text()')
# print(js_script)

txt = js_script[0]
start = txt.find('modules', txt.find('modules')+1) +10
end = index_of_nth(txt, 'column', 4) - 16
json_string = txt[start:end]
# print(json_string)
data = json.loads(json_string)
body = data.get('data').get('body')
content = body.get('content')
p_tags = content[0].get('children')
author_bio = []
for i in range(len(p_tags)):
    author_bio.append(''.join(map(str, p_tags[i].get('children'))))
author_bio = ''.join(author_bio).strip()
# print ("author_bio:", author_bio)

In [ ]:
print(author_bio)

In [ ]:
from bs4 import BeautifulSoup
from lxml import etree
import requests
from FiScrape.items import index_of_nth
import json
  
  
URL = 'https://www.reuters.com/authors/scott-murdoch'
# URL = 'https://www.reuters.com/authors/nelson-renteria/'
  
HEADERS = ({'User-Agent':
            "Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; compatible; Googlebot/2.1; \
            +http://www.google.com/bot.html) Chrome/W.X.Y.Z Safari/537.36",\
            'Accept-Language': 'en-US, en;q=0.5'})
            
webpage = requests.get(URL, headers=HEADERS)
soup = BeautifulSoup(webpage.content, "lxml")
dom = etree.HTML(str(soup))
js_script = dom.xpath('//script[contains(., "window.Fusion")]/text()')
txt = str(js_script)
with open('initial_response2.json', 'w') as  f:
    f.write(txt)
# start = txt.find('description') +len('description')+3
# print(start)
# end = txt[start:].find('",', ) +start-1
# print(end)
# bio_string = txt[start:end]
# print(bio_string.strip())

In [ ]:
from bs4 import BeautifulSoup
from lxml import etree
import requests
from FiScrape.items import index_of_nth
import json
  
  
# URL = 'https://www.reuters.com/authors/scott-murdoch'
# URL = 'https://www.reuters.com/authors/nelson-renteria/'
# URL = 'https://www.reuters.com/authors/jody-godoy/'
# URL = 'https://www.reuters.com/authors/john-mccrank/'
URL = 'https://www.reuters.com/authors/r-mark-halligan/'
  
HEADERS = ({'User-Agent':
            "Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; compatible; Googlebot/2.1; \
            +http://www.google.com/bot.html) Chrome/W.X.Y.Z Safari/537.36",\
            'Accept-Language': 'en-US, en;q=0.5'})
            
webpage = requests.get(URL, headers=HEADERS)
soup = BeautifulSoup(webpage.content, "lxml")
dom = etree.HTML(str(soup))
js_script = dom.xpath('//script[contains(., "window.Fusion")]/text()')
txt = str(js_script)
# with open('initial_response2.json', 'w') as  f:
#     f.write(txt)
start = txt.find('topics') +len('topics')+3
print(start)
end = txt[start:].find('entity', ) +start-3
print(end)
author_string = txt[start:end]
author_string = author_string.replace(r'\\"', '')
print(author_string)
json_author = json.loads(author_string)
# print(json_author.strip())
with open('json_author.json', 'w') as  f:
    f.write(json.dumps(json_author))
if json_author.get('name') == 'Scott Murdoch':
    author_bio = json_author.get('description')
    print (author_bio)